<a href="https://colab.research.google.com/github/SaharZargarzadeh/llm-profile-matcher/blob/main/chatbot_competition_Sahar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AI Agent Expert Matching Competition – Details: 🔍 Competition Overview You’ll be provided with: • An Excel file containing 40 mock professional profiles (including name, job title, contact info, city, and availability) • Corresponding LinkedIn profile HTML files hosted on GitHub 🎯 Your Challenge Build an AI agent with a simple, user-friendly interface that takes user’s queries and intelligently matches them with the most relevant professionals. Your solution should: • Extract Data – Analyze the Excel file and parse LinkedIn HTML files to identify key information such as skills, roles, and experience • Score Matches – Calculate a relevance score (0–10) based on professional background and availability • Explain the Match – Provide a brief, clear explanation for why the selected professional is the best fit • Bonus – Add creative features to enhance the experience and earn extra points Example: A user asks: “I need to discuss an issue with a data expert who can manage a team.” The AI agent analyzes all the rows of Excel and identifies Liam Chen, a Product Manager with 8 years of experience in data strategy and team leadership. The agent returns Liam’s contact information and available time slots along with a matching score.


"""
AI-Powered Professional Matching Agent
--------------------------------------

This application helps match user-defined job or talent queries with realistic professional profiles
using a combination of structured and unstructured data.

🔍 How it works:
1. The user types a natural language query (e.g., "Looking for a senior AI engineer in Chicago with experience in computer vision and team management").
2. The system loads:
   - Structured candidate information from an Excel sheet (Name, Job Title, City, Availability)
   - Unstructured profile content from HTML files (mock LinkedIn profiles)
3. Optional dropdowns (City, Job Title) allow users to apply quick Excel-based filters before AI scoring.
4. For each candidate, the system sends the following to Gemini for scoring:
   - The original user query
   - Candidate metadata (from Excel)
   - Candidate resume text (from HTML)
5. Gemini returns a relevance score (0–10) for each candidate.
6. The top 5 most relevant profiles are shown in the output, including name, job title, city, availability, contact info, and score.

💡 Key Features:
- Natural language query support
- Optional structured filtering (city, job)
- Semantic matching via Gemini API
- Interactive UI using ipywidgets in Google Colab
- API call caching to avoid repeated costs


Author: [Sahar Zargarzadeh]
Competition: AI Agent Expert Matching Challenge 2025
"""



In [ ]:
# Install required packages
!pip install -U google-generativeai pandas beautifulsoup4 scikit-learn

# 📌 Imports
import google.generativeai as genai
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os
import json


In [ ]:
# -----------------------------
# 🔐 Set up Gemini API Key (from Colab secret or directly)
# -----------------------------
from google.colab import userdata
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

# ✅ Then configure the genai SDK
import google.generativeai as genai
genai.configure(api_key=GOOGLE_API_KEY)

from google.colab import files

# Upload your entire ZIP file of the folder
uploaded = files.upload()  # Choose your ZIP file (e.g., mock-linkedin-profiles-main.zip)


Saving mock-linkedin-profiles-main.zip to mock-linkedin-profiles-main.zip


In [ ]:
import zipfile

with zipfile.ZipFile("mock-linkedin-profiles-main.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/mock_profiles")


Promt + some easy filter for excel sheet and then send them to the geminai for scoring

In [ ]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.4 MB/s eta 0:00:00


In [ ]:
# ✅ FULL MATCHING SYSTEM WITH SCORE + EXPLANATION OUTPUT

import os
import pandas as pd
import json
import time
from bs4 import BeautifulSoup
import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import userdata

# -------------------------------
# Step 1: Setup Gemini API
# -------------------------------
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-pro-latest")

# -------------------------------
# Step 2: Load Excel + HTML Profiles
# -------------------------------
excel_path = "/content/mock_profiles/mock-linkedin-profiles-main/mock_profiles_aicompetition.xlsx"
html_folder = "/content/mock_profiles/mock-linkedin-profiles-main/profiles"
df = pd.read_excel(excel_path)

def extract_text_from_html(path):
    with open(path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
        return soup.get_text(separator=" ", strip=True)

html_files = {
    os.path.splitext(file)[0].replace("_", " ").title(): os.path.join(html_folder, file)
    for file in os.listdir(html_folder) if file.endswith(".html")
}

# -------------------------------
# Step 3: Scoring Function with Explanation
# -------------------------------
def score_similarity_gemini(query, metadata, profile_text):
    prompt = f"""
You are helping match professionals to job needs.

Query: {query}

Candidate Metadata:
{metadata}

Candidate Profile:
{profile_text}

1. On a scale of 0 to 10, how relevant is this candidate to the query? Respond as:
Score: <number>

2. In 1–2 sentences, explain why this candidate is a good or bad match.
"""
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()
        lines = text.splitlines()

        # Extract score
        score_line = next((l for l in lines if "Score:" in l), "Score: 0")
        score = float(score_line.split("Score:")[1].strip())

        # Extract explanation
        explanation_index = lines.index(score_line)
        explanation = ""
        for line in lines[explanation_index + 1:]:
            if line.strip():
                explanation = line.strip()
                break

        return score, explanation

    except Exception as e:
        print(f"⚠️ Gemini error: {e}")
        return 0.0, "No explanation (Gemini error)."

# -------------------------------
# Step 4: UI Controls with Editable Query
# -------------------------------
city_options = ['Any'] + sorted(df['City'].dropna().unique().tolist())
role_options = ['Any'] + sorted(df['Job Title'].dropna().unique().tolist())

city_dropdown = widgets.Dropdown(options=city_options, description='City:')
role_dropdown = widgets.Dropdown(options=role_options, description='Job:')
query_input = widgets.Textarea(
    value='Looking for a senior AI engineer experienced with TensorFlow and edge computing',
    placeholder='Type or modify your search query here...',
    description='Query:',
    layout=widgets.Layout(width='100%', height='100px')
)
run_button = widgets.Button(description="Find Matches", button_style='primary')
output_area = widgets.Output()

# Optional: Force-clear old cache to avoid unpacking error
cache_file = "score_cache.json"
if os.path.exists(cache_file):
    os.remove(cache_file)
    print("✅ Old cache cleared.")

# Fresh empty cache
score_cache = {}


# -------------------------------
# Step 5: UI Button Logic
# -------------------------------
def on_button_click(b):
    clear_output(wait=True)
    display(city_dropdown, role_dropdown, query_input, run_button, output_area)

    city = city_dropdown.value
    job = role_dropdown.value
    query = query_input.value.strip()

    print(f"\n🔍 Matching Query: \"{query}\"")
    if city != 'Any': print(f"📍 Filtered City: {city}")
    if job != 'Any': print(f"💼 Filtered Job Title: {job}")

    filtered_df = df.copy()
    if city != 'Any':
        filtered_df = filtered_df[filtered_df['City'].str.lower() == city.lower()]
    if job != 'Any':
        filtered_df = filtered_df[filtered_df['Job Title'].str.lower() == job.lower()]

    profile_texts = {}
    for name in filtered_df['Name']:
        html_path = html_files.get(name)
        if html_path and os.path.exists(html_path):
            profile_texts[name] = extract_text_from_html(html_path)

    results = []
    limited_names = list(profile_texts.keys())[:10]

    for name in limited_names:
        text = profile_texts[name]
        person = filtered_df[filtered_df["Name"] == name].iloc[0]
        metadata = f"Name: {name}\nJob Title: {person['Job Title']}\nCity: {person['City']}\nContact: {person['Contact Number']}\nAvailable Spot: {person['Available Spot']}"
        full_query = f"{query} (Job: {person['Job Title']}, City: {person['City']})"
        cache_key = f"{full_query} | {name}"

        if cache_key in score_cache:
            score, explanation = score_cache[cache_key]
        else:
            score, explanation = score_similarity_gemini(full_query, metadata, text[:2000])
            score_cache[cache_key] = (score, explanation)
            time.sleep(3)

        results.append((name, score, explanation))

    with open(cache_file, "w") as f:
        json.dump(score_cache, f)

    matches = sorted(results, key=lambda x: x[1], reverse=True)
    if not matches:
        print("⚠️ No matches found.")
    else:
        print(f"\n🎯 Top Matches:")
        for name, score, explanation in matches[:5]:
            person = df[df['Name'] == name].iloc[0]
            print(f"\n{name} ({person['Job Title']}, {person['City']})")
            print(f"Score: {score:.2f} | Contact: {person['Contact Number']} | Available Spot: {person['Available Spot']}")
            print(f"📝 Reason: {explanation}")

# -------------------------------
# Step 6: Launch the UI
# -------------------------------
display(city_dropdown, role_dropdown, query_input, run_button, output_area)
run_button.on_click(on_button_click)


Dropdown(description='City:', index=8, options=('Any', 'Ann Arbor, MI', 'Arlington, VA', 'Atlanta, GA', 'Austi…

Dropdown(description='Job:', options=('Any', 'AI Product Manager', 'AI Research Assistant', 'AI Research Scien…

Textarea(value='Looking for a senior AI engineer experienced with TensorFlow and edge computing', description=…

Button(button_style='primary', description='Find Matches', style=ButtonStyle())

Output()


🔍 Matching Query: "Looking for a senior AI engineer experienced with TensorFlow and edge computing"
📍 Filtered City: Chicago, IL


⚠️ Gemini error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.

🎯 Top Matches:

Aisha Khan (HR Consultant, Chicago, IL)
Score: 0.00 | Contact: +1-312-654-9876 | Available Spot: Tuesday 9AM-12PM
📝 Reason: No explanation (Gemini error).


AI Agent Expert Matching System – Dynamic Prompt Version
--------------------------------------------------------

This version of the AI matching agent uses a fully Gemini-driven approach for scoring and reasoning,
without applying any hard filters before calling the LLM.

💬 How It Works:
1. The user submits a natural language query describing their desired professional (e.g., skills, city, availability).
2. The system loads:
   - Candidate metadata from an Excel file (Name, Job Title, City, Contact Info, Availability)
   - Resume-style profile text from corresponding HTML files (LinkedIn mock pages)
3. For each candidate, Gemini receives the full context:
   - The original user query
   - Candidate's structured data (from Excel)
   - Candidate's unstructured profile content (from HTML)
4. Gemini returns:
   - A relevance score (0–10)
   - A short explanation of why this candidate is or isn’t a match
5. The system displays the top-ranked candidates, including contact and availability info.

🧠 Why This Version:
- No dropdown filters or manual Excel constraints
- Gemini handles all interpretation (skills, availability, location, and job fit)
- Maximizes language model reasoning and simplifies the user interface

🎯 Ideal For:
- Open-ended hiring needs
- Users unfamiliar with filtering systems
- Flexible, natural search experience



Author: [Sahar Zargarzadeh]
Submission: AI Agent Expert Matching Competition 2025


dynamic Prompt- every thing handle by geminai


In [ ]:
# ✅ FULL CODE – Dynamic Prompt Version (Gemini Handles Everything)

import os
import pandas as pd
import json
import time
from bs4 import BeautifulSoup
import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import userdata

# -------------------------------
# Step 1: Setup Gemini API
# -------------------------------
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-pro-latest")

# -------------------------------
# Step 2: Load Excel + HTML Profiles
# -------------------------------
excel_path = "/content/mock_profiles/mock-linkedin-profiles-main/mock_profiles_aicompetition.xlsx"
html_folder = "/content/mock_profiles/mock-linkedin-profiles-main/profiles"
df = pd.read_excel(excel_path)

def extract_text_from_html(path):
    with open(path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
        return soup.get_text(separator=" ", strip=True)

html_files = {
    os.path.splitext(file)[0].replace("_", " ").title(): os.path.join(html_folder, file)
    for file in os.listdir(html_folder) if file.endswith(".html")
}
# -------------------------------
# Step 3: Gemini Scoring Function
# -------------------------------
def score_similarity_gemini(query, metadata, profile_text):
    prompt = f"""
You are helping match professionals to job needs based on structured metadata and their profile text.

Query: {query}

Candidate Metadata:
{metadata}

Candidate Profile:
{profile_text}

1. On a scale of 0 to 10, how relevant is this candidate to the query? Respond as:
Score: <number>

2. In 1–2 sentences, explain why this candidate is a good or bad match.
"""
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()
        lines = text.splitlines()

        score_line = next((l for l in lines if "Score:" in l), "Score: 0")
        score = float(score_line.split("Score:")[1].strip())

        explanation_index = lines.index(score_line)
        explanation = ""
        for line in lines[explanation_index + 1:]:
            if line.strip():
                explanation = line.strip()
                break

        return score, explanation

    except Exception as e:
        print(f"⚠️ Gemini error: {e}")
        return 0.0, "No explanation (Gemini error)."

# -------------------------------
# Step 4: UI Elements
# -------------------------------
query_input = widgets.Textarea(
    value='Looking for a professional with experience in computer vision, management, available on Wednesday, and based in Chicago.',
    placeholder='Type your search prompt here...',
    description='Query:',
    layout=widgets.Layout(width='100%', height='100px')
)
run_button = widgets.Button(description="Find Matches", button_style='primary')
output_area = widgets.Output()

# Optional: Force-clear old cache to avoid unpacking error
cache_file = "score_cache.json"
if os.path.exists(cache_file):
    os.remove(cache_file)
    print("✅ Old cache cleared.")

# Fresh empty cache
score_cache = {}


# -------------------------------
# Step 5: Run Match Logic
# -------------------------------
def on_button_click(b):
    clear_output(wait=True)
    display(query_input, run_button, output_area)

    query = query_input.value.strip()
    if not query:
        print("❗ Please enter a query.")
        return

    print(f"\n🔍 Matching Query: \"{query}\"")

    profile_texts = {}
    for name in df['Name']:
        html_path = html_files.get(name)
        if html_path and os.path.exists(html_path):
            profile_texts[name] = extract_text_from_html(html_path)

    results = []
    limited_names = list(profile_texts.keys())[:10]  # Limit to prevent quota exhaustion

    for name in limited_names:
        text = profile_texts[name]
        person = df[df["Name"] == name].iloc[0]

        metadata = f"""Name: {name}
Job Title: {person['Job Title']}
City: {person['City']}
Contact: {person['Contact Number']}
Available Spot: {person['Available Spot']}"""

        cache_key = f"{query} | {name}"
        if cache_key in score_cache:
            score, explanation = score_cache[cache_key]
        else:
            score, explanation = score_similarity_gemini(query, metadata, text[:2000])
            score_cache[cache_key] = (score, explanation)
            time.sleep(3)

        results.append((name, score, explanation))

    with open(cache_file, "w") as f:
        json.dump(score_cache, f)

    matches = sorted(results, key=lambda x: x[1], reverse=True)
    if not matches:
        print("⚠️ No matches found.")
    else:
        print(f"\n🎯 Top Matches:")
        for name, score, explanation in matches[:5]:
            person = df[df['Name'] == name].iloc[0]
            print(f"\n{name} ({person['Job Title']}, {person['City']})")
            print(f"Score: {score:.2f} | Contact: {person['Contact Number']} | Available Spot: {person['Available Spot']}")
            print(f"📝 Reason: {explanation}")

# -------------------------------
# Step 6: Launch Interface
# -------------------------------
display(query_input, run_button, output_area)
run_button.on_click(on_button_click)


Textarea(value='Looking for a professional with experience in computer vision, management, available on Wednes…

Button(button_style='primary', description='Find Matches', style=ButtonStyle())

Output()


🔍 Matching Query: "Looking for a professional with experience in computer vision, management, available on Wednesday, and based in Chicago."


⚠️ Gemini error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


...

⚠️ Gemini error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


⚠️ Gemini error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


⚠️ Gemini error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


⚠️ Gemini error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


⚠️ Gemini error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.

🎯 Top Matches:

Jamal Washington (DevOps Engineer, Atlanta, GA)
Score: 0.00 | Contact: +1-404-876-5432 | Available Spot: Tuesday 1PM-4PM
📝 Reason: No explanation (Gemini error).

Sofia Mendes (Finance Manager, Dallas, TX)
Score: 0.00 | Contact: +1-214-333-9012 | Available Spot: Friday 10AM-1PM
📝 Reason: No explanation (Gemini error).

Aisha Khan (HR Consultant, Chicago, IL)
Score: 0.00 | Contact: +1-312-654-9876 | Available Spot: Tuesday 9AM-12PM
📝 Reason: No explanation (Gemini error).

Nadia Ali (AI Research Scientist, Ann Arbor, MI)
Score: 0.00 | Contact: +1-734-222-3456 | Available Spot: Wednesday 11AM-2PM
📝 Reason: No explanation (Gemini error).

Ezra Kimani